In [1]:
"""" This script makes api calls and get the reicpe information,
cleans and extracts exact data needed and saves the data in csv """

#import libraries
import pandas as pd
import json
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import warnings
import pprint

In [2]:
#make API call
url ='https://api.spoonacular.com/recipes/complexSearch?apiKey&addRecipeInformation=True&cuisine=italian&diet=vegetarian&addRecipeNutrition=True&number=100'
response = requests.get(url).json()
#response

{'results': [{'vegetarian': True,
   'vegan': True,
   'glutenFree': True,
   'dairyFree': True,
   'veryHealthy': True,
   'cheap': False,
   'veryPopular': False,
   'sustainable': False,
   'weightWatcherSmartPoints': 17,
   'gaps': 'no',
   'lowFodmap': False,
   'aggregateLikes': 94,
   'spoonacularScore': 98.0,
   'healthScore': 75.0,
   'creditsText': 'Pick Fresh Foods',
   'license': 'CC BY 3.0',
   'sourceName': 'Pick Fresh Foods',
   'pricePerServing': 414.24,
   'id': 715769,
   'title': 'Broccolini Quinoa Pilaf',
   'readyInMinutes': 30,
   'servings': 2,
   'sourceUrl': 'http://pickfreshfoods.com/broccolini-quinoa-pilaf/',
   'image': 'https://spoonacular.com/recipeImages/715769-312x231.jpg',
   'imageType': 'jpg',
   'nutrition': {'nutrients': [{'name': 'Calories',
      'amount': 625.07,
      'unit': 'kcal',
      'percentOfDailyNeeds': 31.25},
     {'name': 'Fat',
      'amount': 30.69,
      'unit': 'g',
      'percentOfDailyNeeds': 47.22},
     {'name': 'Saturated Fa

In [3]:
#define a recipe dataframe
recipe1_df=pd.DataFrame(columns=['id','title','readyInMinutes','servings','image','dishTypes','analyzedInstructions'])

In [4]:
#extracting the data from the url
for recipe in response['results']:
    id =recipe['id']
    title=recipe['title']
    readyInMinutes=recipe['readyInMinutes']
    servings=recipe['servings']
    image=recipe['image']
    dishTypes =recipe['dishTypes']
    analyzedInstructions=recipe['analyzedInstructions']
    
    #printing all variables
    #print(recipe_id)
    #print(recipe_title)
    #print(recipe_image)
    #print(time_mins)
    #print(servings)
    #print(dish_type)
    
    #save the data in the pandas dataframe
    recipe1_df = recipe1_df.append({'id':id,
                             'title':title,
                             
                              'readyInMinutes':readyInMinutes,
                              'servings':servings,
                              'image':image,
                              'dishTypes':dishTypes,
                              'analyzedInstructions':analyzedInstructions}, ignore_index=True)
    

In [5]:
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,dishTypes,analyzedInstructions
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ..."
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ..."
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[fingerfood, antipasti, starter, snack, appeti...","[{'name': '', 'steps': [{'number': 1, 'step': ..."


In [6]:
recipe1_df['cuisines']='Italian'
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,dishTypes,analyzedInstructions,cuisines
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[fingerfood, antipasti, starter, snack, appeti...","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian


# One hot encoding the DishType column and dropping it

In [7]:
recipe1_df['dish']=0
recipe1_df.columns

Index(['id', 'title', 'readyInMinutes', 'servings', 'image', 'dishTypes',
       'analyzedInstructions', 'cuisines', 'dish'],
      dtype='object')

In [8]:
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,dishTypes,analyzedInstructions,cuisines,dish
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,0
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,0
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[fingerfood, antipasti, starter, snack, appeti...","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,0


In [9]:
meal=[]
for i in range(0,55):
    item =recipe1_df['dishTypes'][i]
    if len(item)==0:
        recipe1_df.iloc[i,8]=0
        
    else:
        #print(item[0])
        
        if any(x in ['lunch', 'dinner', 'antipasti','side dish','salad'] for x in item):
            #print(f"{i}:lunch")
            recipe1_df.iloc[i,8]=1
        if any(x in ['breakfast', 'bread'] for x in item):
            recipe1_df.iloc[i,8]=0
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,dishTypes,analyzedInstructions,cuisines,dish
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[lunch, main course, main dish, dinner]","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[fingerfood, antipasti, starter, snack, appeti...","[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1


In [10]:
recipe1_df.drop(columns=['dishTypes'],inplace=True)
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,analyzedInstructions,cuisines,dish
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1


In [11]:
#Getting ingredients for each recipe
#Ingredients is a list of dictionaries in nutrition key
ingredients_list=[]
recipe_idlist=[]

for recipe in response['results']:
    recipe_id =recipe['id']
    ingres=recipe['nutrition']['ingredients']
    ingri_dict={}
    for ingredient_dict in ingres:
        name=ingredient_dict['name']
        amount=ingredient_dict['amount']
        unit=ingredient_dict['unit']
        value=str(round(amount,2))+' '+unit 
        #key=name+' '+(unit)
        ingri_dict[name]=value
    ingredients_list.append(ingri_dict)
    recipe_idlist.append(recipe_id)    

In [12]:
ingredient_names=[]
for i in range(0,55):
    ingredients=list(ingredients_list[i])
    ingredient_names.append(ingredients)
len(ingredient_names)

55

In [13]:
recipe1_df['ingredients']=ingredients_list

In [14]:
recipe1_df['grocery']=ingredient_names
recipe1_df['grocery'][0]

['broccolini',
 'garlic clove',
 'olive oil',
 'onion',
 'quinoa',
 'vegetable broth',
 'walnuts']

In [15]:
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,analyzedInstructions,cuisines,dish,ingredients,grocery
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1,"{'broccolini': '0.5 bunch', 'garlic clove': '0...","[broccolini, garlic clove, olive oil, onion, q..."
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1,"{'artichoke': '1.0 pound', 'bay leaf': '1.0 ',...","[artichoke, bay leaf, coriander seeds, dried b..."
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,"[{'name': '', 'steps': [{'number': 1, 'step': ...",Italian,1,"{'crusty bread': '0.25 loaf', 'mushrooms': '0....","[crusty bread, mushrooms, spinach, chickpeas, ..."


# Extracting Instructions

In [16]:
recipe_instructions=recipe1_df[['analyzedInstructions','id']]
#recipe_instructions.head(10)

In [17]:
recipe_inst=[]
for instructions,ids in zip(list(recipe_instructions['analyzedInstructions'].values),list(recipe_instructions['id'].values)):
    #print(instructions,ids)
    #print(instructions[0]['steps'])
    instruct={}
    recipe_step=''
    try:
    #break   
        for no_steps in range(len(instructions[0]['steps'])):
            #print(no_steps)
            recipe_step+=instructions[0]['steps'][no_steps]['step']    
            #break
            #print(recipe_step)
        instruct[ids]=recipe_step
        #print(instruct)
        recipe_inst.append(instruct)
    except:
        pass

In [18]:
instruction_final=pd.DataFrame()
for dict_recipe in recipe_inst:
    df=pd.DataFrame(dict_recipe,index=dict_recipe.keys())
    df.columns=['instructions']
    #print(df)
    #break
    instruction_final=pd.concat([instruction_final,df],ignore_index=False,axis=0)

In [19]:
instruction_final.index.names = ['id']
#instruction_final

In [20]:
recipe1_df=recipe1_df.merge(instruction_final,on='id')
recipe1_df.drop(columns=['analyzedInstructions'],inplace=True)

In [21]:
recipe1_df.head(3)

,id,title,readyInMinutes,servings,image,cuisines,dish,ingredients,grocery,instructions
0,715769,Broccolini Quinoa Pilaf,30,2,https://spoonacular.com/recipeImages/715769-31...,Italian,1,"{'broccolini': '0.5 bunch', 'garlic clove': '0...","[broccolini, garlic clove, olive oil, onion, q...",In a large pan with lid heat olive oil over me...
1,648257,Italian Steamed Artichokes,35,1,https://spoonacular.com/recipeImages/648257-31...,Italian,1,"{'artichoke': '1.0 pound', 'bay leaf': '1.0 ',...","[artichoke, bay leaf, coriander seeds, dried b...",Snip the thorns off the artichoke leaves.Place...
2,1095745,Mushroom Hummus Crostini,30,4,https://spoonacular.com/recipeImages/1095745-3...,Italian,1,"{'crusty bread': '0.25 loaf', 'mushrooms': '0....","[crusty bread, mushrooms, spinach, chickpeas, ...","To make the hummus, place all ingredients in a..."


# Saving the first 55 recipes dataframe to a csv

In [22]:
recipe1_df.to_csv('./all_recipes/recipe1.csv',index=False)

In [23]:
ingres1_df=pd.DataFrame(ingredients_list,index= recipe_idlist)
ingres1_df.fillna(value=0,inplace=True)
ingres1_df.index.name='recipe_id'
ingres1_df.head()

,broccolini,garlic clove,olive oil,onion,quinoa,vegetable broth,walnuts,artichoke,bay leaf,coriander seeds,...,powdered sugar,old fashioned rolled oats,chia seeds,strong coffee,greek yogurt,mascarpone cheese,cocoa powder,chocolate & extra cocoa powder,olives,a few sprigs of thyme
recipe_id,,,,,,,,,,,,,,,,,,,,,
715769,0.5 bunch,0.5,0.5 tbsp,0.25 cup,0.5 cup,1.0 cups,1.0 oz,0,0,0,...,0,0,0,0,0,0,0,0,0,0
648257,0,1.0,0,0,0,0,0,1.0 pound,1.0,0.25 tsp,...,0,0,0,0,0,0,0,0,0,0
1095745,0,0,0.25 tbsp,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
658753,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
658515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
#saving the ingredients list of 55 recipes dataframe to csv
ingres1_df.to_csv('./all_ingredients/ingredients1.csv',index=True)

# Nutrition data

In [25]:
nutri_list =[]
id_list=[]
for recipe in response['results']:
    recipe_id =recipe['id']
    nutrients =recipe['nutrition']['nutrients']
    nutri_dict={}
    for nutrient_dict in nutrients:
        name = nutrient_dict['name']
        amount = nutrient_dict['amount']
        unit = nutrient_dict['unit']
        key = name+' '+unit
        #print(name)
        nutri_dict[key]=amount
    nutri_list.append(nutri_dict)
    id_list.append(recipe_id)

In [26]:
#dataframe with nutrinional values of each recipe
nutrition_df= pd.DataFrame(nutri_list,index=id_list)
nutrition_df.index.name='recipe_id'
#nutrition_df

In [27]:
print(nutrition_df.duplicated().sum())

0


In [28]:
#saving the nutrition details of 55 recipes dataframe to csv
nutrition_df.to_csv('./all_nutrition/nutrition1.csv',index=True)